# 4ake

In [1]:
using Printf, DelimitedFiles
using Plots
using DataFrames
using Revise, MDToolbox

In [2]:
function read_ascii(fname)
    lines = readlines(fname)
    natom = length(lines)
    xyz = zeros(Float64, 1, natom*3)
    for i = 1:natom
        xyz[1, 3*(i-1)+1] = parse(Float64, lines[i][31:38])
        xyz[1, 3*(i-1)+2] = parse(Float64, lines[i][39:46])
        xyz[1, 3*(i-1)+3] = parse(Float64, lines[i][47:54])
    end
    TrjArray(xyz=xyz)
end

read_ascii (generic function with 1 method)

In [3]:
fnames = []
omega = []
phi = []
prou = []
d = []
score = []
rmsd = []
struc = []

Any[]

In [4]:
nrun = 1000

1000

In [5]:
for irun = 1:nrun

    for i in 1:100
        filename = "$(irun)/complex.$(i).pdb"
        #t = mdload(filename)
        #t = MDToolbox.readpdb_biostructures(filename)
        t = read_ascii(filename)
        t1 = t[:, 1:1656]
        t2 = t[:, 1657:end]
        r = MDToolbox.compute_skrew(t1, t2)
        push!(fnames, filename)
        push!(omega, r.omega)
        push!(phi, r.phi / (2.0 * pi) * 360.0)
        push!(prou, r.prou)
        push!(d, r.d)
        push!(struc, t)
        #t_ca = t["atomname CA"]
        #push!(rmsd, compute_rmsd_permute(ref, t_ca, 2))
    end

    s = readdlm("$(irun)/zdock.out", skipstart=5)
    score = [score; s[1:100, end]]
end

In [6]:
df = DataFrame(fnames=fnames, score=score, omega=omega, phi=phi, prou=prou, d=d, struc=struc)

,fnames,score,omega,phi,prou
,Any,Any,Any,Any,Any
1,1/complex.1.pdb,1024.65,"[-0.0281216, 0.968563, 0.247175]",142.637,"[-8.75967, 5.99937, -24.5054]"
2,1/complex.2.pdb,995.933,"[0.574083, -0.500062, -0.648356]",147.139,"[-14.8289, 9.28651, -20.2926]"
3,1/complex.3.pdb,975.648,"[0.586679, -0.575964, -0.569274]",159.814,"[-12.8229, 8.26763, -21.5797]"
4,1/complex.4.pdb,962.002,"[-0.135677, -0.937363, 0.320845]",150.089,"[-11.0709, -7.65869, -27.0568]"
5,1/complex.5.pdb,915.738,"[-0.588624, 0.61065, 0.529744]",163.212,"[-12.2797, 7.92458, -22.7795]"
6,1/complex.6.pdb,909.798,"[0.0512852, 0.569022, -0.820722]",93.3045,"[-11.8917, -23.1444, -16.7896]"
7,1/complex.7.pdb,874.365,"[0.235571, -0.971857, -0.000318399]",151.007,"[-15.8386, -3.83192, -22.0794]"
8,1/complex.8.pdb,872.42,"[-0.855621, 0.514046, -0.0605748]",174.23,"[-5.56816, -11.4507, -18.5221]"
9,1/complex.9.pdb,868.374,"[-0.778727, 0.603605, 0.17101]",43.4144,"[20.4586, 30.4552, -14.334]"


In [7]:
df_org = deepcopy(df);

In [8]:
#using BSON: @save, @load
#@save "skrew_parameters.bson" fnames score omega phi prou d df rmsd

In [9]:
#using BSON: @save, @load
#@save "df.bson" df

In [10]:
using JLD2

save("skrew_parameters.jld2", "fnames", fnames, "score", score, "omega", omega, "phi", phi, "prou", prou, "d", d, "df", df, "rmsd", rmsd)
save("df.jld2", "df", df)